In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
#from sklearn.pipeline import Pipelines
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
from keras.layers import Dense,LSTM,Conv2D, BatchNormalization,Flatten, MaxPooling2D
from keras.layers import Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
from keras.layers import Input, Lambda, Reshape, Dropout, Activation

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Dotenv
from dotenv import dotenv_values
# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_resnet import *

In [ ]:
# Define paths
config = dotenv_values(".env")
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']


In [ ]:
# Some constants
DATE_START = '1979-01-01'
DATE_END = '2020-12-31'
# train -test (80/20 of total yy), train-> split 80/20 for validation
#YY_TRAIN = [1979, 2011] # 2005-2011 for validation
#YY_VAL = [2005,2011]
#YY_TEST = [2012, 2020]
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2020]
YY_VALID = 2005
LEVELS = [500, 850, 1000]
G = 9.80665 

# define coordinates (for now use the same for the ppredictors and target)

#LONS = [-25, 30]
#LATS = [30, 75]
# Note: for U-Net I have to select inputs/outputs as power of 2 for an easier implementation(maybe not neccesarily, but I couldn't find a better way)
LONS = [-33, 30]
LATS = [30, 77]


## Data loading

In [ ]:
# define paths for the variables used
G = 9.80665
l_paths = ['/precipitation/day_grid1/','/geopotential/grid1/','/temperature/grid1/','/relative_humidity/day_grid1/',
              '/U_wind/day_grid1/','/V_wind/day_grid1/','/total_column_water/day_grid1/']
v_vars = ['pr', 'z','t2m','rh','u850','v850','tpcw']

In [ ]:
list_vars = load_data(v_vars, l_paths, G, PATH_ERA5, DATE_START, DATE_END, LONS, LATS, LEVELS)

In [ ]:
pr = list_vars[0]
datasets = list_vars[1:7]

In [ ]:
# checking dimensions
print('dimension of pr:',pr.dims)
print('dimension of z',datasets[0].dims)
print('dimension of t2m:',datasets[1].dims)
print('dimension of rh:',datasets[2].dims)
print('dimension of u:',datasets[3].dims)
print('dimension of v:',datasets[4].dims)
print('dimension of tw:',datasets[5].dims)

In [ ]:
# Define precipitation extremes using the 95th
th95 = 0.95 # can also be 0.99
pr95 = precip_exceedance_xarray(pr, th95)

In [ ]:
# Then we need a dictionary for all the variables and levels we want to extract from the dataset
#dic = OrderedDict({'z': 3, 'T2MMEAN':None, 'r':3})
dic = {
     'z': LEVELS,
      'T2MMEAN': None,
      'r': LEVELS,
      'u': None,
      'v': None,
      'tcwv':None}

In [ ]:
# Check if all have the same latitude order
for idat in range(0, len(datasets)):
    # Invert lat axis if needed
    if datasets[idat].lat[0].values < datasets[idat].lat[1].values:
        print('change lat order', idat)
        datasets[idat] = datasets[idat].reindex(lat=list(reversed(datasets[idat].lat)))

In [ ]:
ds = xr.merge(datasets)

In [ ]:
BATCH_SIZE=64 # try increase, decrease it

In [ ]:
# Split into training and test, then I will use DataGenerator class to get the validation
ds_train = ds.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                             '{}-12-31'.format(YY_TRAIN[1])))
ds_test = ds.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                            '{}-12-31'.format(YY_TEST[1])))

dy_train = pr95.sel(time=slice('{}-01-01'.format(YY_TRAIN[0]),
                             '{}-12-31'.format(YY_TRAIN[1])))
dy_test = pr95.sel(time=slice('{}-01-01'.format(YY_TEST[0]),
                             '{}-12-31'.format(YY_TEST[1])))

In [ ]:
# Create a training and validation data generator. Use the train mean and std for validation as well.
dg_train = MyDataGenerator(ds_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                dy_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')), dic, batch_size= BATCH_SIZE, load=True)

In [ ]:
# Create a validation. Use the train mean and std for validation as well. And suffle
dg_val = MyDataGenerator(ds_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                dy_train.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')), dic, batch_size=BATCH_SIZE,  mean=dg_train.mean, std=dg_train.std, load=True)

In [ ]:
# Now also a generator for testing. Impartant: Shuffle must be False!
dg_test = MyDataGenerator(ds_test, dy_test, dic, batch_size=BATCH_SIZE, mean=dg_train.mean, std=dg_train.std, shuffle=False)

In [ ]:
# checks
n_figs = len(dg_train.data[0,0,0,:])
ncols = 5
nrows = -(-n_figs // ncols)
fig, axes = plt.subplots(figsize=(24, 3*nrows), ncols=ncols, nrows=nrows)
for i in range(n_figs):
    i_row = i // ncols
    i_col = i % ncols
    im = axes[i_row, i_col].imshow(np.mean(dg_train.data[:,:,:,i], axis=0))
    fig.colorbar(im, ax=axes[i_row, i_col])

In [ ]:
# Clear session and set tf seed
keras.backend.clear_session()
tf.random.set_seed(42)

# U-Net architecture

In [ ]:
# Define args for the U-net model
i_shape = dg_train.data.shape[1:]
o_shape = dg_train.labels.shape[1:]

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = 1
num_filters = 16
use_batchnorm = True
dropout = True
lr = 0.0004
optimizer = tf.optimizers.Adam(learning_rate = lr)
EPOCHS = 20
# Note: for U-net the input and output must be power of 2 ..
# I will re-define the input for an easier implementation

In [ ]:
METRICS = [
    tf.metrics.CategoricalAccuracy(name='accuracy'),
    tf.metrics.Precision(class_id = 1, name='precision'),
    tf.metrics.Recall(class_id = 1, name='recall')
]

In [ ]:
weights = class_weight.compute_class_weight('balanced', classes = np.unique(dg_train.labels), y = np.array(dg_train.labels).flatten())

In [ ]:
model_loss = weighted_binary_cross_entropy(weights = {0: weights[0].astype('float32'), 1: weights[1].astype('float32')})

In [ ]:
u_mod = Unet2(i_shape, output_channels, num_filters, use_batchnorm, dropout)

In [ ]:
# Build the model and compile
um = u_mod.build_model()
um.compile(optimizer =optimizer, loss = model_loss , metrics = METRICS)

In [ ]:
h_unet = um.fit(dg_train, epochs=EPOCHS, validation_data= dg_val)

In [ ]:
figure, axis = plt.subplots(1, 3)

# For Sine Function
pd.DataFrame(h_unet.history)[['loss','val_loss']].plot(figsize=(18,4), ax=axis[0], grid=True)
pd.DataFrame(h_unet.history)[['precision','val_precision']].plot(figsize=(18,4), ax=axis[1],grid=True)
pd.DataFrame(h_unet.history)[['recall','val_recall']].plot(figsize=(18,4), ax=axis[2],grid=True)
plt.show()



In [ ]:
# make predictions
pred = um.predict(dg_test.data)

In [ ]:
pred= pred.reshape(pred.shape[0],pred.shape[1],pred.shape[2])

In [ ]:
rmse = compute_weighted_rmse(dg_test.labels,pred) #OK, I need to check this out..

In [ ]:
rmse

## ResNet

In [ ]:
m_res = Adapted_resnet(i_shape) # not working..need to check if makes sense..

In [ ]:
m_res.compile(optimizer = optimizer, loss = model_loss , metrics = METRICS)

In [ ]:
h_res = m_res.fit(dg_train, epochs=EPOCHS, validation_data= dg_val)